In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
from wrf import getvar, interplevel, to_np, get_basemap, ll_to_xy,latlon_coords,latlon_coords, smooth2d,get_cartopy, cartopy_xlim, cartopy_ylim, vertcross,CoordPair
import pandas as pd
from netCDF4 import Dataset
import netCDF4
import h5py as h5
import csv
from scipy import stats
import matplotlib as mpl
import glob
import pyproj
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
import seaborn as sns
from metpy.plots import SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
import xarray as xr
from metpy.io import Level2File

In [ ]:
def wrf_pres(P,PB):
    return (P+PB) #in Pa

def wrf_height(PH,PHB):
    return (PH+PHB)/9.81

def wrf_theta(PTEMP):
    return (PTEMP+300.)

def wrf_temp(THETA,PRES):
    return (THETA)*(PRES/1000.)**(.2854) # in hPa
def es(T):
    return (610.7*np.exp(((2500800+(4218-1870.4)*273.16)*(1/273.16-1/T)-(4218-1870.4)*np.log(T/273.16))/461.51))

In [ ]:
filename = "wrfout_d01_2017-01-08_04:20:00"
ncfile = h5.File(filename, 'r')

In [ ]:
lat = ncfile['XLAT'][:].squeeze()
lon = ncfile['XLONG'][:].squeeze()
ncfile.close()

In [ ]:
upper_bound = [2150, 2650, 3150, 3650, 4150, 4650, 5150, 5650, 6150, 6650, 7150] 
lower_bound = [1850, 2350, 2850, 3350, 3850, 4350, 4850, 5350, 5850, 6350, 6850]
alt_namelist = ["2.0km", "2.5km","3.0km","3.5km","4.0km","4.5km","5.0km","5.5km","6.0km","6.5km","7.0km"]
file_list = ["wrfout_d01_2017-01-08_02:20:00","wrfout_d01_2017-01-08_02:30:00","wrfout_d01_2017-01-08_02:40:00","wrfout_d01_2017-01-08_02:50:00","wrfout_d01_2017-01-08_03:00:00","wrfout_d01_2017-01-08_03:10:00","wrfout_d01_2017-01-08_03:20:00","wrfout_d01_2017-01-08_03:30:00","wrfout_d01_2017-01-08_03:40:00","wrfout_d01_2017-01-08_03:50:00","wrfout_d01_2017-01-08_04:00:00","wrfout_d01_2017-01-08_04:10:00","wrfout_d01_2017-01-08_04:20:00","wrfout_d01_2017-01-08_04:30:00","wrfout_d01_2017-01-08_04:40:00","wrfout_d01_2017-01-08_04:50:00","wrfout_d01_2017-01-08_05:00:00","wrfout_d01_2017-01-08_05:10:00","wrfout_d01_2017-01-08_05:20:00","wrfout_d01_2017-01-08_05:30:00","wrfout_d01_2017-01-08_05:40:00","wrfout_d01_2017-01-08_05:50:00"]

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.img_tiles as cimgt
request = cimgt.StamenTerrain()   # very responsive
crg = request.crs
z = 0
u = 0
while z< 600:
    while u < 900:
        new = crg.transform_point(lon[z,u], lat[z,u], crs.Geodetic())
        lon[z,u] = new[0]
        lat[z,u] = new[1]
        u = u + 1
    z = z + 1
    u = 0

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The StamenTerrain class was deprecated in v0.17. Please use Stamen('terrain-background') instead.
  """


In [ ]:
#Define counter for while loop
ct = 0
for file in file_list:
    
    #open file and grab variables for the specific time file
    ncfile = h5.File(file, 'r')
    cloud_mix = ncfile['QCLOUD'][:].squeeze()*1000 #G/KG
    ice_mix = ncfile['QICE'][:].squeeze()*1000 #G/KG
    rain_mix = (ncfile['QRAIN'][:].squeeze() + ncfile['QSNOW'][:].squeeze() + ncfile['QGRAUP'][:].squeeze())*1000
    P = ncfile['P'][:].squeeze()
    PB = ncfile['PB'][:].squeeze()
    PHB = ncfile['PHB'][:].squeeze()
    PH = ncfile['PH'][:].squeeze()
    HEIGHT = wrf_height(PH,PHB)
    ncfile.close()
    ct = 0
    
    #Going to loop through each height level
    while ct < 10:
        
        #Define x and y 0, we will loop throug heach point in the wrf array
        x = 0
        y = 0
        
        #Create the new arrays we will put the values in for a specific height
        cloud = np.zeros((600, 900))
        rain = np.zeros((600, 900))
        ice = np.zeros((600, 900))
        arr = np.zeros((600, 900))
        
        #Loop through each wrf point
        while y < 600:
            while x < 900:
                
                #Find where the height where they are greater than the lower bound, and higher than the upper bound
                #Specified earlier 
                w = np.argwhere(((HEIGHT[:,y,x] >= lower_bound[ct]) & (HEIGHT[:,y,x] <= upper_bound[ct])))
                
                #Determine if the w list is odd or even, pick the middle index
                if len(w) > 0:
                    if len(w)%2 > 0:
                        index = w[int((len(w)-1)/2)]
                    if len(w)%2 == 0:
                        index = w[int(len(w)/2)]
                    cloud[y,x] = cloud_mix[index,y,x]
                    ice[y,x] = ice_mix[index,y,x]
                    rain[y,x] = rain_mix[index,y,x]
                    arr[y,x] = index
                if len(w) == 0:
                    cloud[y,x] = 0
                    ice[y,x] = 0
                    rain[y,x] = 0
                    arr[y,x] = 0
                x = x + 1
            y = y + 1
            x = 0
        
        #Reseting x and y for the next loop
        x = 0
        y = 0
        
        #Make the map
        
        
        def make_map(projection=ccrs.PlateCarree()):
            fig, ax = plt.subplots(figsize=(10, 10),
                                   subplot_kw=dict(projection=projection))
            gl = ax.gridlines(draw_labels=True)
            gl.xlabels_top = gl.ylabels_right = False
            gl.xformatter = LONGITUDE_FORMATTER
            gl.yformatter = LATITUDE_FORMATTER
            return fig, ax

        fig, ax = make_map(projection = crg)

        # specify map extent here
        lonmin, lonmax = -116.70062, -115.44735
        latmin, latmax = 43.756786, 44.655064

        LL = crg.transform_point(lonmin, latmin, ccrs.Geodetic())
        UR = crg.transform_point(lonmax, latmax, ccrs.Geodetic())
        EW = UR[0] - LL[0]
        SN = UR[1] - LL[1]
        side = max(EW,SN)
        mid_x, mid_y = LL[0]+EW/2.0, LL[1]+SN/2.0  #center location

        extent = [mid_x-side/2.0, mid_x+side/2.0, mid_y-side/2.0, mid_y+side/2.0]   # map coordinates, meters

        #Specificy the range of dBz
        ax.set_extent(extent, crs=crg)
        ax.add_image(request, 8)
        cint = .01
        cmin = .05
        cmax = .90
        clevs = np.arange(cmin, cmax, cint)
        iint = 0.05
        imin = 0.01
        imax = 0.3
        rint = 0.05
        rmin = 0.01
        rmax = 0.90
        ilevs = np.arange(imin,imax,iint)
        rlevs = np.arange(rmin,rmax,rint)
        
        #Create the time and name convention
        
        time = file[22:24] + file[25:27]+ "Z"

        picy,picx = crg.transform_point(-115.78, 43.77, crs.Geodetic())
        
        
        title_name = "ERA5"
        
        pic_name = alt_namelist[ct]+ time + "ERA5_Cloud_CAPPI.png"
        
        #Contour!
        cont = ax.contour(lon[269:385,343:456], lat[269:385,343:456], ice[269:385,343:456],ilevs, cmap = "cool", extend = "max" )
        cont2 = ax.contourf(lon[269:385,343:456], lat[269:385,343:456], cloud[269:385,343:456],clevs, cmap=get_cmap("Blues"), extend = "max" )
        cont3 = ax.contour(lon[269:385,343:456], lat[269:385,343:456], rain[269:385,343:456],rlevs, cmap="ocean",extend = "max")
        plt.text(picy,picx, time, fontsize = 23, color = "red")
        plt.title(title_name, fontsize = 30) 
        plt.savefig(pic_name, dpi = 400)
        ct = ct + 1
    ct = 0

NameError: name 'file_list' is not defined